#### Подгрузка  данных

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas

In [15]:
datasets = ['books','authors', 'publishers', 'ratings', 'reviews']
def head_5 (df):
    
    query = '''SELECT * FROM {} LIMIT 5'''.format(df)
    df = pd.io.sql.read_sql(query, con = engine)
    display (df)

In [16]:
for i in datasets:
    head_5(i)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Данные отоброжаются корректно во всех пяти датасетах, приступим к выполнению SQL-запросов.


- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаемколичество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [23]:
query = '''

SELECT COUNT(*) AS books
FROM books
WHERE CAST(publication_date AS TIMESTAMP) >= CAST('2000-01-01' AS TIMESTAMP)

        '''

df = pd.io.sql.read_sql(query, con = engine)
print(df)

   books
0    821


После 1 января 2000 года вышло более 800 книг.

In [24]:
query = '''                              

SELECT b.title AS title,
        COUNT(DISTINCT rw.review_id) AS reviews,
        AVG(r.rating) AS rating
FROM books AS b
INNER JOIN ratings AS r ON r.book_id = b.book_id
LEFT JOIN reviews AS rw ON rw.book_id = b.book_id
GROUP BY b.title,
         b.book_id
ORDER BY reviews DESC

        ''' 
df = pd.io.sql.read_sql(query, con = engine)
print(df.sort_values(by='rating',ascending=False))
print(df.describe().T)

                                                 title  reviews  rating
940               Welcome to Temptation (Dempseys  #1)        2    5.00
300            School's Out—Forever (Maximum Ride  #2)        3    5.00
943  Wherever You Go  There You Are: Mindfulness Me...        2    5.00
709                                              March        2    5.00
710                                        Marvel 1602        2    5.00
..                                                 ...      ...     ...
497  The World Is Flat: A Brief History of the Twen...        3    2.25
685                                              Junky        2    2.00
510                                      Drowning Ruth        3    2.00
659                  His Excellency: George Washington        2    2.00
651                               Harvesting the Heart        2    1.50

[1000 rows x 3 columns]
          count      mean       std  min  25%  50%       75%  max
reviews  1000.0  2.793000  1.074852  0.0  2.0

В среднем на каждую книгу приходится по 3 обзора, а оценка сосатвляет 4.0. Оценку менее 3.5 имеют не больше 250 книг.

In [25]:
query = '''

SELECT p.publisher,
       COUNT(b.book_id) AS books
FROM publishers AS p
JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 5

       '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

                  publisher  books
0             Penguin Books     42
1                   Vintage     31
2  Grand Central Publishing     25
3          Penguin Classics     24
4                    Bantam     19


 Издательство Penguin Books выпустило 42 книги толщиной более 50 страниц.

In [26]:
query = '''

SELECT a.author_id AS id,
       a.author AS author,
       COUNT(r.rating) AS count_rating,
       AVG(r.rating) AS avg_rating
FROM authors AS a
INNER JOIN books AS b ON b.author_id = a.author_id
INNER JOIN ratings AS r ON r.book_id = b.book_id
GROUP BY id,
         author
HAVING COUNT(r.rating) >= 50 
ORDER BY count_rating DESC
LIMIT 1
                        
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

    id                      author  count_rating  avg_rating
0  236  J.K. Rowling/Mary GrandPré           312    4.288462


Самые высокие оценки за свои книги получал J.K. Rowling/Mary GrandPré   

In [29]:
query = '''

WITH T AS (SELECT username,
          COUNT(review_id) as rewiew
          FROM reviews
          WHERE username IN (SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id)>50)
          GROUP BY username)
SELECT ROUND(SUM(rewiew)/COUNT(rewiew), 2) AS avg
FROM T

        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

     avg
0  24.33


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24

#### Вывод и рекомендации

После 1 января 2000 года вышло более 800 книг.
* В среднем на каждую книгу приходится по 3 обзора, а оценка сосатвляет 4.0. Оценку менее 3.5 имеют не больше 250 книг.
* Издательство Penguin Books выпустило 42 книги толщиной более 50 страниц.
* Самые высокие оценки за свои книги получала J.K. Rowling/Mary GrandPré.
* Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.

Рекомендации: 
* Для привлечения новых клиентов можно предоставлять одну из книг автора J.K. Rowling/Mary GrandPré к прочтению бесплатно. Так как автор получал хорошие оценки от действующих клиентов это может стимулировать покупку подписки для дальнейшего чтения.
* Для удержания текущих пользователей можно разработать алгоритм,предлагающий к прочтению книгип , которые высоко оценили другие пользователи.
* Для улучшения сервиса можно также подписать с издательствами, которые выпускают большее количество книг, соглашение, дабы новинки появлялись на сервисе раньше, чем у конкурентов.


 

## Дополнительные запросы

А) Выведем таблицу, которая будет содержать по году публикации:
 количество издательств,
 выпущенных книг и
 сколько всего тысяч страниц было в изданных книгах
 (отобразить только те года, в которых издано более 30 книг)

In [112]:
query = '''

SELECT DATE_TRUNC('year',CAST(b.publication_date AS TIMESTAMP)) AS year,
       COUNT(DISTINCT(b.publisher_id)) AS count_publishers,
       COUNT(b.book_id) AS count_books,
       SUM(b.num_pages)/1000 AS sum_pages
FROM books AS b 
GROUP BY DATE_TRUNC('year',CAST(b.publication_date AS TIMESTAMP))
HAVING COUNT(b.book_id) > 30 
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

        year  count_publishers  count_books  sum_pages
0 1999-01-01                26           41         15
1 2000-01-01                35           38         13
2 2001-01-01                41           60         21
3 2002-01-01                62           94         38
4 2003-01-01                65          105         41
5 2004-01-01                88          124         46
6 2005-01-01                89          139         55
7 2006-01-01               109          184         68
8 2007-01-01                38           50         18


Б) Выведем в одной таблице два числа — среднюю оценку тех книг, на которые написало отзывов более 3 человек и отдельно среднюю оценку остальных книг, сделаем выводы какой рейтинг больше

In [63]:
query = '''

SELECT AVG(rating)
          FROM ratings AS r
          WHERE r.book_id IN(SELECT rw.book_id
                             FROM reviews AS rw
                             GROUP BY rw.book_id
                             HAVING COUNT(DISTINCT(rw.username))>3)

UNION ALL

SELECT AVG(rating)
          FROM ratings AS r
          WHERE r.book_id IN(SELECT rw.book_id
                             FROM reviews AS rw
                             GROUP BY rw.book_id
                             HAVING COUNT(DISTINCT(rw.username))<=3)


        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

        avg
0  3.949505
1  3.890191


Средний рейтинг книг на которые написало отзывов более 3 человек выше, чем у  остальных

В) Выведем топ пять пользователей по суммарному показателю написанных ревью и поставленных оценок

In [106]:
query = '''

WITH D AS (SElECT r.username AS name,
                  rw.review_id AS review,
                  r.rating_id AS rating
           FROM ratings r
           JOIN reviews AS rw ON r.username=rw.username
           )
SELECT name,
       COUNT(DISTINCT(review)) + COUNT(DISTINCT(rating)) AS sum

FROM D
GROUP BY name
ORDER BY sum DESC
LIMIT 5
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

             name  sum
0     sfitzgerald   83
1      martinadam   83
2       richard89   81
3  jennifermiller   78
4          paul88   78
